In [ ]:
import torch

x = torch.randn(2, 3)
mask = torch.tensor([True, False, True])  # shape (3,)
x.masked_fill(mask, 0)  # ❌ 错，因为 (3,) 无法广播到 (2, 3)

a = torch.rand((3, 5))
print(a)
print(a.unsqueeze(1))
print(a.unsqueeze(1).unsqueeze(-1))

In [ ]:
import torch



N, num_head, q_seq_len, k_seq_len = 1, 2, 5, 5
mul_res = torch.tensor([[
[
    [1.2, 0.3, 0.4, 0.2, 0.1],  # token 0
    [0.3, 1.0, 0.6, 0.5, 0.1],  # token 1
    [0.4, 0.6, 1.3, 0.7, 0.2],  # token 2
    [0.2, 0.5, 0.7, 1.1, 0.4],  # token 3
    [0.1, 0.1, 0.2, 0.4, 1.5]   # token 4 (padding)
],
[
    [0.4, 0.6, 1.3, 0.7, 0.2],  
    [0.2, 0.5, 0.7, 1.1, 0.4],  
    [0.1, 0.1, 0.2, 0.4, 1.5],
    [1.2, 0.3, 0.4, 0.2, 0.1],  
    [0.3, 1.0, 0.6, 0.5, 0.1],  
]
]])
# print(mul_res.shape)    # (N, num_head, q_len, k_len)
key_padding_mask = torch.tensor([[False, False, False, False, True]])  # 只有最后一个 token 是 padding
# print(key_padding_mask)
attn_mask = torch.triu(torch.ones((q_seq_len, k_seq_len)), diagonal=1).bool()   # (q_len=5, k_len=5)

if key_padding_mask is not None:
    key_padding_mask = key_padding_mask.unsqueeze(1).unsqueeze(2)
    key_padding_mask = key_padding_mask.expand((N, num_head, q_seq_len, k_seq_len))
    mul_res = torch.masked_fill(mul_res, key_padding_mask, -torch.inf)
print(mul_res.shape)
print(mul_res)
if attn_mask is not None:
    attn_mask = attn_mask.unsqueeze(0).unsqueeze(0)
    attn_mask = attn_mask.expand((N, num_head, q_seq_len, k_seq_len))
    mul_res = torch.masked_fill(mul_res, attn_mask, -torch.inf)
print(mul_res.shape)
print(mul_res)

# key_padding_mask = key_padding_mask.unsqueeze(1).unsqueeze(2)
# key_padding_mask = key_padding_mask.expand((1, num_head, 5, 5))
# print(key_padding_mask)
# print(torch.softmax(torch.tensor([float("inf"), float("inf")]), dim=0))
# attn_output_weights = attn_output_weights.masked_fill(, float('-inf'))
# print(attn_output_weights)

In [ ]:
from collections import OrderedDict
from transformers import AutoModel
from transformers import BertModel

ckp = "google-bert/bert-base-uncased"

bert = AutoModel.from_pretrained(ckp)
print(bert)

In [5]:
import torch
import torch.nn.functional as F
import torch.nn as nn

inputs = torch.randn((2, 3))
inputs = torch.softmax(inputs, dim=0)
labels = torch.tensor([0, 2])
print(nn.CrossEntropyLoss()(inputs, labels))
print(F.cross_entropy(inputs, labels))
print(F.cross_entropy(inputs[0], labels[0]))

print(inputs[:, 0:2].shape)


tensor(0.8358)
tensor(0.8358)
tensor(0.8265)
torch.Size([2, 2])


In [2]:
import torch
import torch.nn as nn

# t = torch.arange(24).view(2, 3, 4)
# print(t)
# t = t.view(2, 4, 3)
# print(t)


a = torch.arange(12)
print(torch.stack([a] * 3, dim=0))

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]])


In [1]:
import torch

def top_p(data: torch.Tensor, p):
    max_k = data.shape[-1]
    k = 2000
    top_k, indices = torch.topk(data, k)    # top_k has been sorted by descending order
    cumsum = torch.cumsum(top_k, dim=-1)
    if not torch.all(cumsum[:, -1] > p).item():     # if k is not large enough to cover cumulated probability p 
        top_k, indices = torch.topk(data, max_k)
        cumsum = torch.cumsum(top_k, dim=-1)
    sorted_cum_topk, sorted_cum_indices = torch.sort(cumsum, dim=-1, descending=True) 
    targets = torch.searchsorted(sorted_cum_topk, p)
    bs, max_len = sorted_cum_topk.shape
    positions = torch.arange(max_len).unsqueeze(0).expand(bs, -1)
    top_k = top_k.masked_fill(positions > targets, 0)
    samples_idx = torch.multinomial(top_k, 1)
    res_indices = torch.gather(indices, -1, samples_idx)
    res_values = torch.gather(data, -1, res_indices)
    return res_values, res_indices


t = torch.randn((2, 10))
v, i = top_p(t, 0.6)
# print(t)
# print(v)
# print(i)



RuntimeError: selected index k out of range

In [2]:
import torch
from MyTransformers import MyMultiHeadAttention
from torch.nn import MultiheadAttention

input = torch.randn((4, 5, 764))
tm = MultiheadAttention(768, 12, batch_first=True)
print(tm.state_dict())

OrderedDict([('in_proj_weight', tensor([[ 0.0426, -0.0327,  0.0439,  ..., -0.0429, -0.0016,  0.0211],
        [-0.0132, -0.0017,  0.0009,  ...,  0.0218, -0.0096,  0.0073],
        [ 0.0427, -0.0374, -0.0036,  ..., -0.0209,  0.0304, -0.0140],
        ...,
        [-0.0309, -0.0217,  0.0379,  ...,  0.0281,  0.0344, -0.0168],
        [ 0.0157,  0.0332,  0.0440,  ..., -0.0429,  0.0207,  0.0346],
        [-0.0140, -0.0269,  0.0143,  ..., -0.0124, -0.0055, -0.0067]])), ('in_proj_bias', tensor([0., 0., 0.,  ..., 0., 0., 0.])), ('out_proj.weight', tensor([[-0.0180,  0.0349, -0.0099,  ..., -0.0081,  0.0224,  0.0336],
        [-0.0252,  0.0219,  0.0020,  ..., -0.0066, -0.0307,  0.0253],
        [-0.0333, -0.0226,  0.0165,  ...,  0.0206, -0.0189,  0.0132],
        ...,
        [ 0.0216, -0.0233, -0.0103,  ..., -0.0149,  0.0137,  0.0055],
        [-0.0287,  0.0069, -0.0328,  ..., -0.0105, -0.0038, -0.0154],
        [ 0.0281, -0.0262,  0.0110,  ..., -0.0336,  0.0359,  0.0253]])), ('out_proj.bias', 

In [2]:
# fix_copy_embeddings.py
import torch
from transformers import GPT2LMHeadModel
from gpt2 import GPT2, GPT2Config   # 根据你的路径调整导入

DEVICE = "cpu"
hf = GPT2LMHeadModel.from_pretrained("gpt2").to(DEVICE).eval()
cfg = hf.config

# instantiate your model (same config)
mycfg = GPT2Config(hidden_size=cfg.n_embd, vocab_size=cfg.vocab_size,
                   num_hidden_layer=cfg.n_layer, num_head=cfg.n_head, max_len=cfg.n_positions)
my = GPT2(mycfg).to(DEVICE).eval()

# make sure embedding parameter shapes match
print("HF wte shape:", tuple(hf.transformer.wte.weight.shape))
print("My wte shape:", tuple(my.embedding.word_embedding.weight.shape))
print("HF wpe shape:", tuple(hf.transformer.wpe.weight.shape))
print("My wpe shape:", tuple(my.embedding.position_embedding.weight.shape))

# explicit copy
with torch.no_grad():
    # copy token embedding
    my.embedding.word_embedding.weight.data.copy_(hf.transformer.wte.weight.data.to(my.embedding.word_embedding.weight.device))
    # copy position embedding
    my.embedding.position_embedding.weight.data.copy_(hf.transformer.wpe.weight.data.to(my.embedding.position_embedding.weight.device))

# tie lm_head to embedding if you expect that (do this AFTER copying if you reassign)
try:
    # If your model ties at init, this is probably already the case. If not, set it here:
    my.lm_head.weight = my.embedding.word_embedding.weight
except Exception:
    pass

# verify exact equality (should be zero)
diff_wte = (my.embedding.word_embedding.weight.detach().cpu() - hf.transformer.wte.weight.detach().cpu()).abs()
diff_wpe = (my.embedding.position_embedding.weight.detach().cpu() - hf.transformer.wpe.weight.detach().cpu()).abs()
print("After copy: wte mean_abs:", float(diff_wte.mean().item()), "max:", float(diff_wte.max().item()))
print("After copy: wpe mean_abs:", float(diff_wpe.mean().item()), "max:", float(diff_wpe.max().item()))

# quick element samples to sanity-check
print("wte sample diff [0,:8]:", diff_wte[0, :8])
print("wpe sample diff [0,:8]:", diff_wpe[0, :8])


HF wte shape: (50257, 768)
My wte shape: (50257, 768)
HF wpe shape: (1024, 768)
My wpe shape: (1024, 768)
After copy: wte mean_abs: 0.0 max: 0.0
After copy: wpe mean_abs: 0.0 max: 0.0
wte sample diff [0,:8]: tensor([0., 0., 0., 0., 0., 0., 0., 0.])
wpe sample diff [0,:8]: tensor([0., 0., 0., 0., 0., 0., 0., 0.])


In [4]:
# deep_stack_debug.py
import torch, math, sys
from transformers import GPT2LMHeadModel
from gpt2 import GPT2, GPT2Config  # 根据你的文件名调整 import
torch.set_printoptions(sci_mode=False, linewidth=200)

DEVICE = "cpu"
HF_NAME = "gpt2"
TEXT = "Hello, my dog is cute"

def mae(a,b): return (a.detach().float()-b.detach().float()).abs().mean().item()
def maestat(a,b):
    a=a.detach().cpu().float(); b=b.detach().cpu().float()
    d=(a-b).abs()
    return {"mean_abs":float(d.mean().item()), "max_abs":float(d.max().item()), "shape":tuple(a.shape)}

# ---------- load models ----------
hf = GPT2LMHeadModel.from_pretrained(HF_NAME).to(DEVICE).eval()
cfg = hf.config
mycfg = GPT2Config(hidden_size=cfg.n_embd, vocab_size=cfg.vocab_size,
                   num_hidden_layer=cfg.n_layer, num_head=cfg.n_head, max_len=cfg.n_positions)
my = GPT2(mycfg).to(DEVICE).eval()

# ---------- robust copy of core weights (embeddings + per-layer) ----------
print("Copying HF weights to my model (robust) ...")
hfsd = hf.state_dict()
with torch.no_grad():
    # embeddings
    my.embedding.word_embedding.weight.data.copy_(hfsd["transformer.wte.weight"].to(my.embedding.word_embedding.weight.device))
    my.embedding.position_embedding.weight.data.copy_(hfsd["transformer.wpe.weight"].to(my.embedding.position_embedding.weight.device))
    # tie lm_head if present
    try:
        my.lm_head.weight = my.embedding.word_embedding.weight
    except Exception:
        pass

    # per-layer
    for i in range(cfg.n_layer):
        # layernorms
        copy_map = [
            (f"transformer.h.{i}.ln_1.weight", my.layers[i].layernorm1.weight),
            (f"transformer.h.{i}.ln_1.bias",   my.layers[i].layernorm1.bias),
            (f"transformer.h.{i}.ln_2.weight", my.layers[i].layernorm2.weight),
            (f"transformer.h.{i}.ln_2.bias",   my.layers[i].layernorm2.bias),
        ]
        for k,dst in copy_map:
            if k in hfsd:
                dst.data.copy_(hfsd[k].to(dst.device))

        # c_attn (qkv) : canonicalize and split
        W = hfsd[f"transformer.h.{i}.attn.c_attn.weight"]
        b = hfsd[f"transformer.h.{i}.attn.c_attn.bias"]
        d_model = cfg.n_embd
        if W.shape[0] == 3*d_model and W.shape[1]==d_model:
            Wcanon = W
        elif W.shape[0] == d_model and W.shape[1] == 3*d_model:
            Wcanon = W.t().contiguous()
        else:
            raise RuntimeError(f"Unexpected c_attn.weight shape: {W.shape}")
        Wq,Wk,Wv = Wcanon.split(d_model, dim=0)
        bq,bk,bv = b.split(d_model, dim=0)
        my.layers[i].self_attn.q_w.weight.data.copy_(Wq.to(my.layers[i].self_attn.q_w.weight.device))
        my.layers[i].self_attn.q_w.bias.data.copy_(bq.to(my.layers[i].self_attn.q_w.bias.device))
        my.layers[i].self_attn.k_w.weight.data.copy_(Wk.to(my.layers[i].self_attn.k_w.weight.device))
        my.layers[i].self_attn.k_w.bias.data.copy_(bk.to(my.layers[i].self_attn.k_w.bias.device))
        my.layers[i].self_attn.v_w.weight.data.copy_(Wv.to(my.layers[i].self_attn.v_w.weight.device))
        my.layers[i].self_attn.v_w.bias.data.copy_(bv.to(my.layers[i].self_attn.v_w.bias.device))

        # attn out proj (c_proj)
        Wcproj = hfsd[f"transformer.h.{i}.attn.c_proj.weight"]
        bcproj = hfsd[f"transformer.h.{i}.attn.c_proj.bias"]
        # copy orientation-robustly
        dstW = my.layers[i].self_attn.linear.weight
        if tuple(Wcproj.shape) == tuple(dstW.shape):
            dstW.data.copy_(Wcproj.to(dstW.device))
        elif tuple(Wcproj.t().contiguous().shape) == tuple(dstW.shape):
            dstW.data.copy_(Wcproj.t().contiguous().to(dstW.device))
        else:
            raise RuntimeError(f"unexpected c_proj shape {Wcproj.shape} vs {dstW.shape}")
        my.layers[i].self_attn.linear.bias.data.copy_(bcproj.to(my.layers[i].self_attn.linear.bias.device))

        # mlp: c_fc -> linear1 (HF often (d,4d)), c_proj -> linear2 (HF often (4d,d))
        W_fc = hfsd[f"transformer.h.{i}.mlp.c_fc.weight"]
        b_fc = hfsd[f"transformer.h.{i}.mlp.c_fc.bias"]
        dst_fc = my.layers[i].fnn.linear1.weight
        # try direct or transpose
        if tuple(W_fc.shape) == tuple(dst_fc.shape):
            dst_fc.data.copy_(W_fc.to(dst_fc.device))
        elif tuple(W_fc.t().contiguous().shape) == tuple(dst_fc.shape):
            dst_fc.data.copy_(W_fc.t().contiguous().to(dst_fc.device))
        else:
            raise RuntimeError(f"unexpected mlp.c_fc shape {W_fc.shape} vs {dst_fc.shape}")
        my.layers[i].fnn.linear1.bias.data.copy_(b_fc.to(my.layers[i].fnn.linear1.bias.device))

        W_proj = hfsd[f"transformer.h.{i}.mlp.c_proj.weight"]
        b_proj = hfsd[f"transformer.h.{i}.mlp.c_proj.bias"]
        dst_proj = my.layers[i].fnn.linear2.weight
        if tuple(W_proj.shape) == tuple(dst_proj.shape):
            dst_proj.data.copy_(W_proj.to(dst_proj.device))
        elif tuple(W_proj.t().contiguous().shape) == tuple(dst_proj.shape):
            dst_proj.data.copy_(W_proj.t().contiguous().to(dst_proj.device))
        else:
            raise RuntimeError(f"unexpected mlp.c_proj shape {W_proj.shape} vs {dst_proj.shape}")
        my.layers[i].fnn.linear2.bias.data.copy_(b_proj.to(my.layers[i].fnn.linear2.bias.device))
print("Copy finished.\n")

# ---------- parameter equality check ----------
print("Checking per-layer parameter equality (q/k/v, c_proj, mlp) ...")
for i in range(cfg.n_layer):
    # check q/k/v weights exactly equal
    hW = hfsd[f"transformer.h.{i}.attn.c_attn.weight"]
    if hW.shape[0] == cfg.n_embd and hW.shape[1] == 3*cfg.n_embd:
        hWcanon = hW.t().contiguous()
    elif hW.shape[0] == 3*cfg.n_embd and hW.shape[1] == cfg.n_embd:
        hWcanon = hW
    else:
        raise RuntimeError("unexpected shape")
    Wq,Wk,Wv = hWcanon.split(cfg.n_embd, dim=0)
    # convert to CPU for comparison
    errors = []
    def chk(name, hf_tensor, my_tensor):
        hf = hf_tensor.detach().cpu()
        myt = my_tensor.detach().cpu()
        m = (hf-myt).abs().max().item()
        if m != 0.0:
            errors.append((name, m))
    chk(f"h{i}.q_w", Wq, my.layers[i].self_attn.q_w.weight)
    chk(f"h{i}.k_w", Wk, my.layers[i].self_attn.k_w.weight)
    chk(f"h{i}.v_w", Wv, my.layers[i].self_attn.v_w.weight)
    # c_proj
    chk(f"h{i}.c_proj", hfsd[f"transformer.h.{i}.attn.c_proj.weight"], my.layers[i].self_attn.linear.weight)
    # mlp
    chk(f"h{i}.c_fc", hfsd[f"transformer.h.{i}.mlp.c_fc.weight"], my.layers[i].fnn.linear1.weight)
    chk(f"h{i}.c_proj_mlp", hfsd[f"transformer.h.{i}.mlp.c_proj.weight"], my.layers[i].fnn.linear2.weight)
    if errors:
        print("Layer", i, "param mismatches (max abs):", errors)
        print("-> STOPPING: fix these first.")
        sys.exit(1)
print("All layer params match exactly (max_abs == 0).\n")

# ---------- create inputs and compute hf hidden states ----------
from transformers import GPT2TokenizerFast
tok = GPT2TokenizerFast.from_pretrained(HF_NAME)
ids = tok(TEXT, return_tensors="pt")["input_ids"].to(DEVICE)
attn_mask = torch.ones_like(ids, dtype=torch.long).to(DEVICE)

with torch.no_grad():
    hf_out = hf(ids, attention_mask=attn_mask, output_hidden_states=True, return_dict=True)
hf_hiddens = hf_out.hidden_states  # (embedding, after layer0, after layer1, ...)

# ---------- 1) per-layer compare: apply my.layers[i] on hf_hiddens[i] (i.e. same input) ----------
print("Per-layer single-apply compare (feed HF's layer-input into my layer) ...")
for i in range(cfg.n_layer):
    hf_in = hf_hiddens[i].detach().clone()
    my_block = my.layers[i]
    hf_block = hf.transformer.h[i]
    # apply my_layer to the HF input
    with torch.no_grad():
        my_out = my_block(hf_in, hf_in, hf_in, torch.ones_like(ids))
    # compare with HF's post-layer output
    hf_post = hf_hiddens[i+1].detach().clone()
    s = maestat(hf_post, my_out)
    print(f"Layer {i}: MAE between hf_post and my_out = {s['mean_abs']:.6e}  max={s['max_abs']:.6e}")
    if s["mean_abs"] != 0.0:
        print(">> Layer", i, "differs when applying my_block to HF input. Run detailed_block_debug on this layer.")
        # we stop early to do detailed debug next
        break
else:
    print("All single-layer applications matched HF outputs exactly.\n")

# ---------- 2) now do the full stack on my side (my.embedding -> my.layers sequential) ----------
print("Running full stacked forward on my model and comparing per-layer outputs ...")
with torch.no_grad():
    my_embed = my.embedding(ids)
    h = my_embed
    my_hiddens = []
    # check if any in-place mutation occurs: keep copy of h before call and compare after
    for i,layer in enumerate(my.layers):
        # detect inplace: clone input
        input_clone = h.detach().clone()
        out = layer(h, h, h, torch.ones_like(ids))
        # if the original input changed, that's in-place mutation
        diff_inplace = (input_clone - h).abs().max().item()
        if diff_inplace != 0.0:
            print(f"Layer {i}: detected in-place modification of input! max change = {diff_inplace}")
        my_hiddens.append(out)
        h = out

# compare stacked my_hiddens with hf_hiddens[1:]
for i,(hf_h,my_h) in enumerate(zip(hf_hiddens[1:], my_hiddens)):
    s = maestat(hf_h, my_h)
    print(f"Stacked Layer {i} hidden mae={s['mean_abs']:.6e} max={s['max_abs']:.6e}")
    if s["mean_abs"] != 0.0:
        print(">> Difference appears at stacked layer", i, " (first mismatch).")
        # attempt a fine-grained check: feed HF input into my layer and compare (we already did that above)
        break

print("\nDeep stack debug finished.")


Copying HF weights to my model (robust) ...
Copy finished.

Checking per-layer parameter equality (q/k/v, c_proj, mlp) ...


RuntimeError: The size of tensor a (3072) must match the size of tensor b (768) at non-singleton dimension 1

In [2]:
import torch
import torch.nn as nn

class mynn(nn.Module):
    def __init__(self):
        super().__init__()
        self.hh = nn.Parameter(torch.ones((90,30)))

aa = mynn()
print(aa)
print(torch.softmax(torch.ones((6)) * -1e-8, dim=-1))

mynn()
tensor([0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667])
